## Notebook to download CUDEM tiles

https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/. 

CUDEM tiles will be clipped to the extent of the study area and used to retrieve elevations for each network node.

In [1]:
### Packages

import os
import glob
import requests
import urllib
import json


from osgeo import gdal, ogr
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask

In [2]:
### Set working directory

path='..' # introduce path to your working directory
os.chdir(path)


In [3]:
### Check if extent of raster tile and Pamlico shp overlap. 
#If true, clip raster using polygon and save it 

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

# Create folder if it does no exist
outdir= './data/CUDEM/CUDEM_Clip_Kinston'
if not os.path.exists(outdir):
    os.makedirs(outdir)

for filename1 in os.listdir('./data/CUDEM/Tiles'):
    if filename1.endswith('.tif'):
        raster_dir=('./data/CUDEM/Tiles/{0}'.format(filename1))
        raster_name=filename1.replace ('.tif', '')
        raster = gdal.Open(raster_dir)
        
        # get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize
        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('./data/Kinston'):
            if filename2.endswith('.shp'):
                vector_dir=('./data/Kinston/{0}'.format(filename2))
                vector_name=filename2.replace('.shp', '')
                vector = ogr.Open(vector_dir)
                
                # get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()
                
                # check if they intersect and if they do clip raster tile using polygon
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                        # output clipped raster
                        out_tif = os.path.join('./data/CUDEM/CUDEM_Clip_Kinston/{0}_{1}.tif'.format(vector_name,raster_name))
                        # read the data
                        data = rasterio.open(raster_dir)
                        barrier = gpd.read_file(vector_dir)
                        # project the Polygon into same CRS as the grid
                        barrier = barrier.to_crs(crs=data.crs.data)
                        coords = getFeatures(barrier)
                        # clip raster with polygon
                        out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
                        # copy the metadata
                        out_meta = data.meta.copy()
                        out_meta.update({"driver": "GTiff",
                                        "height": out_img.shape[1],
                                        "width": out_img.shape[2],
                                        "transform": out_transform,
                                        "crs": "EPSG:4269"})
                        # write clipped raster to disk
                        with rasterio.open(out_tif, "w", **out_meta) as dest:
                            dest.write(out_img)       
                else:
                    continue

/home/ebgoldstein/anaconda3/envs/roads/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/ebgoldstein/anaconda3/envs/roads/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
## With the clipped rasters, create CUDEM mosaic for the Davis.shp 

# Create folder if it does no exist
outdir= './data/CUDEM/CUDEM_Mosaic_Kinston'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Merge all clipped rasters that start with the same name (belong to the same barrier) in one mosaic
for vector in os.listdir('./data/Kinston'):
            if vector.endswith('.shp'):
                vector_name= vector.replace('.shp', '')
                # list for the source files
                src_files_to_mosaic = []
                for raster in os.listdir('./data/CUDEM/CUDEM_Clip_Kinston'):
                            if raster.startswith(vector_name):
                                src = rasterio.open('./data/CUDEM/CUDEM_Clip_Kinston/{0}'.format(raster))
                                src_files_to_mosaic.append(src)
                                # merge function returns a single mosaic array and the transformation info
                                mosaic, out_trans = merge(src_files_to_mosaic)
                                # copy the metadata
                                out_meta = src.meta.copy()
                                # update the metadata
                                out_meta.update({"driver": "GTiff",
                                                 "height": mosaic.shape[1],
                                                 "width": mosaic.shape[2],
                                                 "transform": out_trans,
                                                 "crs": "EPSG:4269"})
                                # write the mosaic raster to disk
                                with rasterio.open("./data/CUDEM/CUDEM_Mosaic_Kinston/{0}.tif".format(vector_name), "w", **out_meta) as dest:
                                    dest.write(mosaic)
                                
                            else:
                                continue